<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h1 class="list-group-item list-group-item-action active" data-toggle="list" style='background:#005097; border:0' role="tab" aria-controls="home"><center>Hyperparameter Optimization with Optuna</center></h1>

Optuna is an open-source hyperparameter optimization framework that automates the process of tuning machine learning models by efficiently searching for the best hyperparameters. It supports both classical machine learning models and deep learning models, and its goal is to maximize or minimize a given **objective function**, such as model performance, by optimizing hyperparameters.

In the Optuna optimization framework, a **study** is the main object that represents an optimization session. It encompasses the entire optimization process — including the definition of the objective function, the trials (each set of hyperparameters tested), and the history and results of those trials.

Optuna selects the **sampling strategy** based on the **study configuration** and **hyperparameter types**. The choice depends on the search space, the number of trials, and whether prior knowledge is available.

## 1️⃣ Default Strategy: Tree-structured Parzen Estimator (TPE)
- If no specific sampler is specified, Optuna **automatically** uses **TPE (Tree-structured Parzen Estimator)**.
- TPE **models the probability distribution** of good and bad hyperparameter choices and chooses new trials accordingly.
- Best for **non-convex search spaces** where grid/random search fails.

Example of instantiating a study that uses TPE:

```python
import optuna
study = optuna.create_study(direction="maximize")  # Uses TPE by default

## 2️⃣ Explicitly Specifying a Sampler
You can override the default and choose a specific sampler:

| Sampler                  | When to Use?                                           | Example                                      |
|--------------------------|------------------------------------------------------|----------------------------------------------|
| **TPE (default)**        | Works well in most cases, adaptive Bayesian optimization | `optuna.samplers.TPESampler()`               |
| **Random Search**        | Good for benchmarking, large search spaces          | `optuna.samplers.RandomSampler()`           |
| **Grid Search**          | If you have limited trials and want exhaustive search | `optuna.samplers.GridSampler(search_space)` |
| **CMA-ES**              | Good for continuous spaces, often used in reinforcement learning | `optuna.samplers.CmaEsSampler()` |

Example of choosing a sampler:

```python
import optuna

sampler = optuna.samplers.RandomSampler()  # Choose random search
study = optuna.create_study(direction="maximize", sampler=sampler)

## 3️⃣ How Optuna Adapts to the Problem
Optuna adjusts the search strategy based on:

- Discrete vs. Continuous Parameters. If parameters are categorical (trial.suggest_categorical), TPE handles them well. If parameters are continuous (trial.suggest_float), CMA-ES or TPE works better.
- Log-scaled vs. Linear Search Space. If log=True is used (e.g., learning_rate), Optuna adjusts the sampling distribution accordingly.
- Early Pruning and Convergence. If trials are pruned early, TPE focuses on exploiting promising areas rather than random exploration.

## 4️⃣ Customizing the Sampling Strategy
You can combine samplers or switch strategies mid-experiment:

```python
sampler = optuna.samplers.TPESampler(n_startup_trials=10)  # Use random search for first 10 trials
study = optuna.create_study(sampler=sampler)


# Demo

In [1]:
# pip install optuna

In [8]:
import pandas as pd

data = pd.read_csv('../data/aug_train.csv')

# Split features and target
X = data.drop(columns=['id', 'Response'])
y = data['Response']

# Define categorical and numerical features
categorical_features = ['Gender', 'Vehicle_Age', 'Vehicle_Damage']
numerical_features = X.columns.difference(categorical_features)

In [10]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import optuna

# Preprocessing pipeline
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(), categorical_features)
])

N_TRIALS = 10
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
outer_scores = []

for train_idx, test_idx in outer_cv.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    def objective(trial):
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                random_state=42
            ))
        ])

        inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        scores = cross_val_score(model, X_train, y_train, cv=inner_cv, scoring='f1', n_jobs=-1)
        return np.mean(scores)

    # Run Optuna for current outer fold
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=N_TRIALS)

    # Train model with best params on full inner training set
    best_params = study.best_params
    final_model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(**best_params, random_state=42))
    ])
    final_model.fit(X_train, y_train)
    y_pred = final_model.predict(X_test)
    score = f1_score(y_test, y_pred)
    outer_scores.append(score)

print("Nested CV F1 scores:", outer_scores)
print("Mean F1 score:", np.mean(outer_scores))


[I 2025-04-10 05:27:13,275] A new study created in memory with name: no-name-74b16ce9-45fa-4e35-ace1-193434f61b06


[I 2025-04-10 05:27:26,358] Trial 0 finished with value: 0.3414115867516296 and parameters: {'n_estimators': 69, 'max_depth': 14, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.3414115867516296.
[I 2025-04-10 05:27:58,037] Trial 1 finished with value: 0.00027945307348764586 and parameters: {'n_estimators': 239, 'max_depth': 7, 'min_samples_split': 3, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.3414115867516296.
[I 2025-04-10 05:28:30,901] Trial 2 finished with value: 0.27134730746322805 and parameters: {'n_estimators': 191, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.3414115867516296.
[I 2025-04-10 05:29:12,136] Trial 3 finished with value: 0.06669601151876058 and parameters: {'n_estimators': 285, 'max_depth': 9, 'min_samples_split': 5, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.3414115867516296.
[I 2025-04-10 05:29:55,640] Trial 4 finished with value: 0.07699580221032808 and paramete

Nested CV F1 scores: [0.33155859440677016, 0.414046590175334, 0.4091045823724055, 0.3943604413567634, 0.4289385637395141]
Mean F1 score: 0.39560175441015744


In [11]:
%%time

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Train final model with best hyperparameters
best_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        # max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        random_state=42
    ))
])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train and evaluate the model
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Evaluate performance
f1 = f1_score(y_test, y_pred)
print("F1-score on test set:", f1)

F1-score on test set: 0.4440074991997805
CPU times: user 48.3 s, sys: 75 ms, total: 48.4 s
Wall time: 47.3 s


In [6]:
# Train and evaluate the model
default_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

default_model.fit(X_train, y_train)
y_pred = default_model.predict(X_test)
# Evaluate performance
f1 = f1_score(y_test, y_pred)
print("F1-score on test set:", f1)

F1-score on test set: 0.43990114580993034
